# Data compilation from database scan

The trained models have been used for scaning the whole database looking for incentives.\
Each sentence has been firs classified into one of to categories, *Incentive* or *Not-Incentive*.\
Sentences which were classified as incentives were further classified into one of the six policy instruments.

Thus 
* Number of sentences per document
* Average length of sentences
* Length of smallest sentence
* Length of longest sentence
* Relative distance of the first incentive from the head of the document
* Distance to the closest previous incentive


In [1]:
import boto3
import csv
import json

In [2]:
# path = 'C:/Users/jordi/Documents/claus/'
path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [3]:
def sentence_stats(sentence_text, min_sentence_length, max_sentence_length, sum_sentence_length):
    sum_sentence_length = sum_sentence_length + len(sentence_text)
    if len(sentence_text) < min_sentence_length:
        min_sentence_length = len(sentence_text)
    if len(sentence_text) > max_sentence_length:
        max_sentence_length = len(sentence_text)
    return min_sentence_length, max_sentence_length, sum_sentence_length


# def sentences_processing()    

In [6]:
language = "english"

in_prefix = f"{language}_documents/HSSC/sentences/"
out_prefix = f"{language}_documents/HSSC/updated_sentences/"

i = 0
documents_with_incentives = 0
results_list = [["file_name", "n_incentives", "Supplies", "Loan", "Fine", "Direct payment", "Technical assistance", "Tax benefit"]]
incentives_dict = {}

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):#.:
    if ".json" in obj.key and "ing" not in obj.key:
        i += 1
    #     if i % 10 == 0:
    #       output.clear()
        if i < 100000:
    #         print(f"\n\nFile number: {i}\n\n")
            print(i,  end='\r')
            name = obj.key.split("/")[3].split("_")[0]
            flag = False
            sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
            incentive_counter = 0
            sentences_counter = 0
            min_sentence_length = 100000
            max_sentence_length = 0
            sum_sentence_length = 0
            last_sentence_position = 0
            incentives_dict = {'Supplies': 0,
                               'Loan': 0,
                               'Fine': 0,
                               'Direct payment': 0,
                               'Technical assistance': 0,
                               'Tax benefit': 0}
            for sentence_id in sentences[name]["sentences"]:
                sentences_counter += 1
                sentence = sentences[name]["sentences"][sentence_id]["text"]
                min_sentence_length, max_sentence_length, sum_sentence_length = sentence_stats(sentence, min_sentence_length, max_sentence_length, sum_sentence_length)
                incentive = sentences[name]["sentences"][sentence_id]["label"]  
                if len(incentive) > 1 and incentive[0] == "Incentive":
                    incentive_counter += 1
                    sentence_number = sentence_id.split("_")[2]
                    policy = sentences[name]["sentences"][sentence_id]["label"][1]
                    if last_sentence_position == 0:
                        results_list.append([])
                        last_sentence_position = sentence_number
                        sentences[name]["sentences"][sentence_id]["closest_sentence"] = 0
                        flag = True
                    else:
                        sentences[name]["sentences"][sentence_id]["closest_sentence"] = int(sentence_number) - int(last_sentence_position)
                        last_sentence_position = sentence_number

                    relative_position = int(sentence_number) / sentences[name]["metadata"]["n_sentences"]
                    sentences[name]["sentences"][sentence_id]["relative_position"] = round(relative_position, 2)
                    incentives_dict[policy] += 1
    #                 print(incentive, "\n", sentence_id, " ** ", last_sentence_position)

            # Updating the dictionary
            sentences[name]["metadata"]["min_sent_length"] = min_sentence_length
            sentences[name]["metadata"]["max_sent_length"] = max_sentence_length
            sentences[name]["metadata"]["avg_sent_length"] = int(sum_sentence_length/sentences_counter)
            sentences[name]["metadata"]["number_of_incentives"] = incentive_counter
            for key, value in incentives_dict.items():
                sentences[name]["metadata"][key] = value
            s3.Object(s3BucketName, f"{out_prefix}{name}_sents.json").put(Body = (json.dumps(sentences, indent = 4)))
            # Creating the results list
            if flag:
                documents_with_incentives += 1
                results_list[documents_with_incentives].append(name)
                results_list[documents_with_incentives].append(incentive_counter)
                for key, value in incentives_dict.items():
                    results_list[documents_with_incentives].append(value)

with open(f"../out/incentives_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)

In [89]:
incentives_dict

{'Supplies': 3,
 'Loan': 0,
 'Fine': 3,
 'Direct payment': 1,
 'Technical assistance': 3,
 'Tax benefit': 0}

In [90]:
results_list

[['file_name',
  'n_incentives',
  'Supplies',
  'Loan',
  'Fine',
  'Direct payment',
  'Technical assistance',
  'Tax benefit'],
 ['00009282cf7af3ed1a9e25e1b43791837e7c144a', 2, 1, 0, 0, 1, 0, 0],
 ['002c53058e85d383b057fa4cc25a6eb8e7d401e3', 14, 0, 2, 8, 4, 0, 0],
 ['003a0d20014ce9eec85c61ca05de4454ff7c6321', 10, 3, 0, 3, 1, 3, 0]]

In [91]:
sentences

{'003a0d20014ce9eec85c61ca05de4454ff7c6321': {'metadata': {'n_sentences': 326,
   'language': 'spanish',
   'min_sent_length': 27,
   'max_sent_length': 96696,
   'avg_sent_length': 489,
   'number_of_incentives': 10,
   'Supplies': 3,
   'Loan': 0,
   'Fine': 3,
   'Direct payment': 1,
   'Technical assistance': 3,
   'Tax benefit': 0},
  'sentences': {'003a0d20014ce9eec85c61ca05de4454ff7c6321_sent_0': {'text': 'ACUERDO por el que se da a conocer el resumen del Programa de Manejo del Parque Nacional El Tepozteco ACUERDO por el que se da a conocer el resumen del Programa de Manejo del Parque Nacional El Tepozteco.',
    'label': ['not_Incentive']},
   '003a0d20014ce9eec85c61ca05de4454ff7c6321_sent_1': {'text': 'Al margen un sello con el Escudo Nacional, que dice: Estados Unidos Mexicanos- Secretaría de Medio Ambiente y Recursos Naturales.',
    'label': ['not_Incentive']},
   '003a0d20014ce9eec85c61ca05de4454ff7c6321_sent_2': {'text': 'JUAN RAFAEL ELVIRA QUESADA, Secretario de Medio Am